<a href="https://colab.research.google.com/github/BhaveshNikam09/Deep_learning_practise/blob/main/dog_cat.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
import kagglehub
import os

# Download latest version
path = kagglehub.dataset_download("bhavikjikadara/dog-and-cat-classification-dataset")

print("Path to dataset files:", path)
print(os.listdir(path))

Path to dataset files: /kaggle/input/dog-and-cat-classification-dataset
['PetImages']


In [4]:
import os
import numpy as np
from sklearn.model_selection import train_test_split

dataset_path = "/kaggle/input/dog-and-cat-classification-dataset/PetImages"

categories=['Dog','Cat']

image_path=[]
labels=[]

for category in categories:
  class_path=os.path.join(dataset_path,category)
  images=os.listdir(class_path)

  for image in images:
    image_path.append(os.path.join(class_path,image))
    labels.append(0 if category == "Cat" else 1)

labels=np.array(labels)

train_path,test_path,train_labels,test_labels=train_test_split(image_path,labels,test_size=0.2,random_state=42)

print(f"Total images: {len(image_path)}")
print(f"Train images: {len(train_path)}, Test images: {len(test_path)}")

Total images: 24998
Train images: 19998, Test images: 5000


In [ ]:
import cv2
import numpy as np

# Function to load and normalize images in small batches
def load_and_normalize_images(image_paths, target_size=(224, 224), batch_size=1000):
    images = []
    for i, path in enumerate(image_paths):
        img = cv2.imread(path)
        if img is None:
            print(f"⚠️ Skipping corrupt/missing image: {path}")
            continue
        img = cv2.resize(img, target_size)
        img = img / 255.0
        images.append(img)

        # Process in small batches to avoid crashes
        if (i + 1) % batch_size == 0:
            print(f"✅ Processed {i+1}/{len(image_paths)} images...")

    return np.array(images)

# Load and normalize train & test images in batches
X_train = load_and_normalize_images(train_path, batch_size=500)
X_test = load_and_normalize_images(test_path, batch_size=500)

print(f"Train shape: {X_train.shape}, Test shape: {X_test.shape}")


⚠️ Skipping corrupt/missing image: /kaggle/input/dog-and-cat-classification-dataset/PetImages/Cat/9171.jpg
✅ Processed 500/19998 images...
✅ Processed 1000/19998 images...
⚠️ Skipping corrupt/missing image: /kaggle/input/dog-and-cat-classification-dataset/PetImages/Cat/8470.jpg
⚠️ Skipping corrupt/missing image: /kaggle/input/dog-and-cat-classification-dataset/PetImages/Dog/10158.jpg
✅ Processed 1500/19998 images...
✅ Processed 2000/19998 images...
✅ Processed 2500/19998 images...
✅ Processed 3000/19998 images...
⚠️ Skipping corrupt/missing image: /kaggle/input/dog-and-cat-classification-dataset/PetImages/Dog/7969.jpg
✅ Processed 3500/19998 images...
⚠️ Skipping corrupt/missing image: /kaggle/input/dog-and-cat-classification-dataset/PetImages/Dog/3288.jpg
✅ Processed 4000/19998 images...
⚠️ Skipping corrupt/missing image: /kaggle/input/dog-and-cat-classification-dataset/PetImages/Cat/850.jpg
✅ Processed 4500/19998 images...
✅ Processed 5000/19998 images...
⚠️ Skipping corrupt/missing i

In [6]:
import pandas as pd

# Create DataFrame for training data
train_df = pd.DataFrame({"filename": train_path, "label": train_labels})
test_df = pd.DataFrame({"filename": test_path, "label": test_labels})

# Check the structure
print(train_df.head())


                                            filename  label
0  /kaggle/input/dog-and-cat-classification-datas...      0
1  /kaggle/input/dog-and-cat-classification-datas...      0
2  /kaggle/input/dog-and-cat-classification-datas...      1
3  /kaggle/input/dog-and-cat-classification-datas...      0
4  /kaggle/input/dog-and-cat-classification-datas...      1


In [ ]:
# Convert numeric labels to string labels
train_df["label"] = train_df["label"].astype(str)
test_df["label"] = test_df["label"].astype(str)


In [31]:
import cv2
import numpy as np
from PIL import Image

def check_and_fix_images(image_paths):
    problematic_images = []

    for file_path in image_paths:
        try:
            # Open image with PIL
            with Image.open(file_path) as img:
                img = img.convert("RGB")  # Ensure RGB format
                img_array = np.array(img)  # Convert to NumPy array

                # Check if image array is empty (corrupt image)
                if img_array is None or img_array.size == 0:
                    print(f"Empty image detected: {file_path}")
                    problematic_images.append(file_path)

        except Exception as e:
            print(f"Problematic image found: {file_path} - {e}")
            problematic_images.append(file_path)

    return problematic_images

# Run the function again
problematic_files = check_and_fix_images(train_path)
print(f"Total problematic images: {len(problematic_files)}")


Total problematic images: 0


In [32]:
for file in problematic_files:
    os.remove(file)

print("Corrupt images removed successfully!")


Corrupt images removed successfully!


In [33]:
import pandas as pd

# Create a DataFrame with image paths and labels
train_df = pd.DataFrame({'filename': train_path, 'label': train_labels})
test_df = pd.DataFrame({'filename': test_path, 'label': test_labels})

# Convert numerical labels to strings (required by flow_from_dataframe)
train_df['label'] = train_df['label'].astype(str)
test_df['label'] = test_df['label'].astype(str)

# Create data generators
train_generator = datagen.flow_from_dataframe(
    dataframe=train_df,
    x_col="filename",
    y_col="label",
    target_size=(224, 224),
    batch_size=32,
    class_mode='binary'
)

test_generator = datagen.flow_from_dataframe(
    dataframe=test_df,
    x_col="filename",
    y_col="label",
    target_size=(224, 224),
    batch_size=32,
    class_mode='binary'
)


Found 19998 validated image filenames belonging to 2 classes.
Found 5000 validated image filenames belonging to 2 classes.


In [34]:
from tensorflow.keras.applications import VGG16
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D, Input
from tensorflow.keras.models import Model

# Load the VGG16 model without the top layer
base_model = VGG16(weights='imagenet', include_top=False, input_shape=(224, 224, 3))


for layer in base_model.layers:
    layer.trainable = False


inputs = Input(shape=(224, 224, 3))

x = base_model(inputs, training=False)
x = GlobalAveragePooling2D()(x)
x = Dense(128, activation='relu')(x)
x = Dense(64, activation='relu')(x)


outputs = Dense(1, activation='sigmoid')(x)


model = Model(inputs=inputs, outputs=outputs)


model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])


model.summary()


Model: "functional_4"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ input_layer_12 (InputLayer)          │ (None, 224, 224, 3)         │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ vgg16 (Functional)                   │ (None, 7, 7, 512)           │      14,714,688 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ global_average_pooling2d_4           │ (None, 512)                 │               0 │
│ (GlobalAveragePooling2D)             │                             │                 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_12 (Dense)                     │ (None, 128)                 │          65,664 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_13 (Dense)                     │ (None, 64)                  │           8,256 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_14 (Dense)                     │ (None, 1)                   │              65 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 14,788,673 (56.41 MB)

 Trainable params: 73,985 (289.00 KB)

 Non-trainable params: 14,714,688 (56.13 MB)

In [35]:
history=model.fit(train_generator,epochs=50,validation_data=test_generator,batch_size=16)

Epoch 1/50


/usr/local/lib/python3.11/dist-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


 58/625 ━━━━━━━━━━━━━━━━━━━━ 1:39 175ms/step - accuracy: 0.6413 - loss: 0.6210

/usr/local/lib/python3.11/dist-packages/PIL/TiffImagePlugin.py:949: UserWarning: Truncated File Read
  warnings.warn(str(msg))


625/625 ━━━━━━━━━━━━━━━━━━━━ 163s 256ms/step - accuracy: 0.8298 - loss: 0.3589 - val_accuracy: 0.9238 - val_loss: 0.1905
Epoch 2/50
625/625 ━━━━━━━━━━━━━━━━━━━━ 138s 220ms/step - accuracy: 0.9210 - loss: 0.1908 - val_accuracy: 0.9116 - val_loss: 0.2190
Epoch 3/50
625/625 ━━━━━━━━━━━━━━━━━━━━ 138s 221ms/step - accuracy: 0.9218 - loss: 0.1847 - val_accuracy: 0.9272 - val_loss: 0.1771
Epoch 4/50
625/625 ━━━━━━━━━━━━━━━━━━━━ 138s 221ms/step - accuracy: 0.9267 - loss: 0.1696 - val_accuracy: 0.9130 - val_loss: 0.2143
Epoch 5/50
625/625 ━━━━━━━━━━━━━━━━━━━━ 138s 221ms/step - accuracy: 0.9340 - loss: 0.1586 - val_accuracy: 0.9252 - val_loss: 0.1775
Epoch 6/50
625/625 ━━━━━━━━━━━━━━━━━━━━ 138s 221ms/step - accuracy: 0.9331 - loss: 0.1573 - val_accuracy: 0.9258 - val_loss: 0.1844
Epoch 7/50
625/625 ━━━━━━━━━━━━━━━━━━━━ 138s 221ms/step - accuracy: 0.9355 - loss: 0.1549 - val_accuracy: 0.9174 - val_loss: 0.2170
Epoch 8/50
625/625 ━━━━━━━━━━━━━━━━━━━━ 138s 221ms/step - accuracy: 0.9378 - loss: 0.14

In [38]:
img=cv2.imread("/content/download.jpeg")
img=cv2.resize(img,(224,224))
img=img/255.0
img=np.expand_dims(img,axis=0)
prediction=model.predict(img)

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 51ms/step


In [40]:
label = 1 if prediction >= 0.5 else 0  # Convert probability to class

if label==1:
  print("Dog")
else:
  print("Cat")

Predicted Class: 0
Cat


In [43]:
img=cv2.imread("/content/images.jpeg")
img=cv2.resize(img,(224,224))
img=img/255.0
img=np.expand_dims(img,axis=0)
prediction=model.predict(img)

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 45ms/step


In [44]:
label = 1 if prediction >= 0.5 else 0  # Convert probability to class

if label==1:
  print("Dog")
else:
  print("Cat")

Cat


In [45]:
img=cv2.imread("/content/download (1).jpeg")
img=cv2.resize(img,(224,224))
img=img/255.0
img=np.expand_dims(img,axis=0)
prediction=model.predict(img)
label = 1 if prediction >= 0.5 else 0  # Convert probability to class

if label==1:
  print("Dog")
else:
  print("Cat")

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 44ms/step
Dog


In [48]:
cv2.imread('/content/download (1).jpeg')
prediction=model.predict(img)
label = 1 if prediction >= 0.5 else 0  # Convert probability to class

if label==1:
  print("Dog")
else:
  print("Cat")

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 47ms/step
Dog
